In [1]:
import numpy
import sympy
import sympy.codegen.rewriting

#import pysundials_cffi
#from pysundials_cffi import lambdify

In [16]:
def dydt(y, theta):
    S, X = y
    mu_max, K_S, Y_XS = theta
    dXdt = mu_max * S / (K_S + S)
    return [
        -Y_XS * dXdt,
        dXdt
    ]

def optimize(system):
    """Applies expression optimization on elements in a matrix."""
    def optimizer(expr):
        return sympy.codegen.rewriting.optimize(expr, sympy.codegen.rewriting.optims_c99)
    return system.applyfunc(optimizer)

class ODESystem:
    def __init__(self, dydt, *, n_states, n_theta):
        # set up symbolic represenations of state and parameter vectors
        self.y = sympy.MatrixSymbol('y', n=n_states, m=1)
        self.theta = sympy.MatrixSymbol('theta', n=n_theta, m=1)
        self.v = sympy.MatrixSymbol('v', n_states, 1)
        # pass symbolic vectors through the user-provided ode function and
        # obtain optimized sympy expressions for all matrices that we'll need
        self.odes = optimize(sympy.Matrix(dydt(self.y, self.theta)))
        self.J = optimize(self.odes.jacobian(self.y))
        self.Jv = optimize(self.odes.jacobian(self.y) * sympy.Matrix(self.v))
        self.dy_dtheta = optimize(self.odes.jacobian(self.theta))
        # TODO: compile numba functions for the expressions above
        # TODO: wrap numba functions with cffi to make them act on sundials data types
    

In [17]:
system = ODESystem(dydt, n_states=2, n_theta=3)
system.odes

Matrix([
[-theta[0, 0]*theta[2, 0]*y[0, 0]/(theta[1, 0] + y[0, 0])],
[             theta[0, 0]*y[0, 0]/(theta[1, 0] + y[0, 0])]])

In [18]:
system.J

Matrix([
[-theta[0, 0]*theta[2, 0]/(theta[1, 0] + y[0, 0]) + theta[0, 0]*theta[2, 0]*y[0, 0]/(theta[1, 0] + y[0, 0])**2, 0],
[                         theta[0, 0]/(theta[1, 0] + y[0, 0]) - theta[0, 0]*y[0, 0]/(theta[1, 0] + y[0, 0])**2, 0]])

In [19]:
system.Jv

Matrix([
[(-theta[0, 0]*theta[2, 0]/(theta[1, 0] + y[0, 0]) + theta[0, 0]*theta[2, 0]*y[0, 0]/(theta[1, 0] + y[0, 0])**2)*v[0, 0]],
[                         (theta[0, 0]/(theta[1, 0] + y[0, 0]) - theta[0, 0]*y[0, 0]/(theta[1, 0] + y[0, 0])**2)*v[0, 0]]])

In [20]:
system.dy_dtheta

Matrix([
[-theta[2, 0]*y[0, 0]/(theta[1, 0] + y[0, 0]), theta[0, 0]*theta[2, 0]*y[0, 0]/(theta[1, 0] + y[0, 0])**2, -theta[0, 0]*y[0, 0]/(theta[1, 0] + y[0, 0])],
[             y[0, 0]/(theta[1, 0] + y[0, 0]),            -theta[0, 0]*y[0, 0]/(theta[1, 0] + y[0, 0])**2,                                            0]])